# Common DataFrame manipulations to all storage options 

In [27]:
import os
from openpyxl import load_workbook
import pandas as pd
from datetime import timedelta

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
INTERIM_DATA_DIR = os.path.join(DATA_DIR, 'interim')

INPUT_LOAD_AND_VRE_FILENAME = 'load_and_vre.csv'
FINAL_LOAD_AND_VRE_FILENAME = 'final_load_and_vre.csv'

In [8]:
# all input columns as variables for later use
date_colname = 'date'
load_colname = 'load_mw'
solar_gen_colname = 'solar_mw'
wind_gen_colname = 'wind_mw'

load_and_vre_df = pd.read_csv(os.path.join(RAW_DATA_DIR, LOAD_AND_VRE_FILENAME))
input_cols = [date_colname, load_colname, wind_gen_colname, solar_gen_colname]
load_and_vre_df = load_and_vre_df.iloc[:, [i for i in range(len(input_cols))]]
load_and_vre_df.columns = input_cols

In [44]:
# all derived columns as variables for later use

### COMMON - columns needed bt legacy gen, batteries, hydrogen
total_vre_gen_colname = 'total_vre_mw'
day_of_month_colname = 'day_of_month'
day_of_yr_colname = 'day_of_yr'
week_of_yr_colname = 'wk_of_yr'
month_colname = 'month'
year_colname = 'year'
weekday_colname = 'weekday'
daily_load_vre_diff_colname = 'daily_load_vre_diff'
prev_7_load_colname = 'prev_7_load'
prev_7_load_vre_diff_colname = 'prev_7_load_vre_diff'
prev_7_vre_gen_colname = 'prev_7_vre_gen'
critical_load_mw_colname = 'critical_load_mw'
critical_load_less_vre_colname = 'critical_load_less_vre_mw'
surplus_vre_colname = 'surplus_vre'

### BATTERIES 
cuml_load_since_prev_charge_colname = 'cuml_load_since_prev_charge_mw'
cuml_charge_since_prev_charge_colname = 'cuml_charge_since_prev_discharge_mw'
battery_soc_colname = 'battery_soc'

### HYDROGEN
req_hydrogen_kg_colname = 'reqd_hydrogen_kg'
hydrogen_production_kg_colname = 'hydrogen_prod_kg'
hydrogen_h2o_demand_l_colname = 'hydrogen_h2o_demand_l'
cuml_hydrogen_demand_kg_colname = 'cuml_hydrogen_demand_kg'
cuml_h2_energy_demand_mwh_colname = 'cuml_h2_energy_demand_mwh'

### BAU - LEGACY GENERATION
thermal_gen_total_mw_colname = 'thermal_gen_total_mw'
thermal_gen_rice_mw_colname = 'thermal_gen_rice_mw'
thermal_gen_combustion_mw_colname = 'thermal_gen_combustion_mw'

In [72]:
# other values/assumptions needed for the analysis
CRITICAL_LOAD_PERC = (1/3)
SOLAR_SCALE_FACTOR = 4
WIND_SCALE_FACTOR = .4

In [73]:
def clean_input_columns(df):

    cleaned_df = df.copy()
    cleaned_df = cleaned_df.dropna(how='all')

    # all solar vals need to be > 0
    cleaned_df[solar_gen_colname] = cleaned_df[solar_gen_colname]\
        .apply(lambda x: x if x > 0 else 0)
    
    return cleaned_df

In [74]:
cleaned_load_and_vre_df = clean_input_columns(load_and_vre_df)

In [75]:
def add_common_derived_columns(df):
    
    decorated_df = df.copy()
    
    decorated_df[solar_gen_colname] = decorated_df[solar_gen_colname] * SOLAR_SCALE_FACTOR
    decorated_df[wind_gen_colname] = decorated_df[wind_gen_colname] * WIND_SCALE_FACTOR
    
    decorated_df[total_vre_gen_colname] = \
        decorated_df[wind_gen_colname] + decorated_df[solar_gen_colname]
    
    decorated_df[critical_load_mw_colname] = \
        decorated_df[load_colname] * CRITICAL_LOAD_PERC
    
    decorated_df[critical_load_less_vre_colname] = \
        decorated_df[critical_load_mw_colname] - decorated_df[total_vre_gen_colname]
    
    decorated_df[surplus_vre_colname] = \
        -decorated_df[critical_load_less_vre_colname]

    decorated_df[critical_load_less_vre_colname] = decorated_df[critical_load_less_vre_colname]\
        .map(lambda x: x if x > 0 else 0)

    decorated_df[surplus_vre_colname] = decorated_df[surplus_vre_colname]\
        .map(lambda x: x if x > 0 else 0)
    
    return decorated_df

In [76]:
final_load_and_vre_df = add_common_derived_columns(cleaned_load_and_vre_df)
final_load_and_vre_df.to_csv(os.path.join(INTERIM_DATA_DIR, FINAL_LOAD_AND_VRE_FILENAME), index=False)

# Hydrogen-specific analysis

In [77]:
ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG = 19.99
ELEC_EFF_ELECTROLYSIS_KWH_PER_KG = 55.8

ENERGY_CONSUMPTION_H2O_DESAL_KWH_PER_M3 = 1.8
DENSITY_H2_300BAR_KG_PER_M3 = 20
DENSITY_H2_30BAR_KG_PER_M3 = 2.38
H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3 = 1.4

def calc_h2_cols(df):
    
    decorated_df = df.copy()
    
    decorated_df[req_hydrogen_kg_colname] = \
        (decorated_df[critical_load_less_vre_colname] * 1000)/ ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG
    
    decorated_df[hydrogen_production_kg_colname] = \
        (decorated_df[surplus_vre_colname] * 1000) / ELEC_EFF_ELECTROLYSIS_KWH_PER_KG
    
    decorated_df[hydrogen_h2o_demand_l_colname] = decorated_df[hydrogen_production_kg_colname] * H2O_ELECTROLYSIS_CONSUMPTION_L_PER_M3
    
    
    decorated_df.sort_values(date_colname)
    
## calc cuml h2 demand
    decorated_df[cuml_hydrogen_demand_kg_colname] = None
    for idx, row in decorated_df.iterrows():
        
        if idx == 0:
            val = row[req_hydrogen_kg_colname]
        else : #
            val = \
                decorated_df.loc[idx - 1, cuml_hydrogen_demand_kg_colname] \
                + row[req_hydrogen_kg_colname]\
                - row[hydrogen_production_kg_colname]
        decorated_df.at[idx, cuml_hydrogen_demand_kg_colname] = val
        
    decorated_df[cuml_h2_energy_demand_mwh_colname] = \
        (decorated_df[cuml_hydrogen_demand_kg_colname] * ELEC_EFF_H2_FUEL_CELL_KWH_PER_KG) / 1000
    
    return decorated_df

In [78]:
h2_hourly_df = calc_h2_cols(final_load_and_vre_df)

In [79]:
h2_hourly_df

,date,load_mw,wind_mw,solar_mw,total_vre_mw,critical_load_mw,critical_load_less_vre_mw,surplus_vre,reqd_hydrogen_kg,hydrogen_prod_kg,hydrogen_h2o_demand_l,cuml_hydrogen_demand_kg,cuml_h2_energy_demand_mwh
0,1/1/17 0:00,35.1,0.671864,0.0,0.671864,11.700000,11.028136,0.0,551.682641,0.0,0.0,551.683,11.0281
1,1/1/17 1:00,35.1,0.671864,0.0,0.671864,11.700000,11.028136,0.0,551.682641,0.0,0.0,1103.37,22.0563
2,1/1/17 2:00,34.6,0.493308,0.0,0.493308,11.533333,11.040025,0.0,552.277405,0.0,0.0,1655.64,33.0963
3,1/1/17 3:00,34.2,0.436772,0.0,0.436772,11.400000,10.963228,0.0,548.435618,0.0,0.0,2204.08,44.0595
4,1/1/17 4:00,33.8,0.586040,0.0,0.586040,11.266667,10.680627,0.0,534.298483,0.0,0.0,2738.38,54.7402
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,12/31/17 19:00,45.5,2.669000,0.0,2.669000,15.166667,12.497667,0.0,625.195931,0.0,0.0,180081,3599.82
8756,12/31/17 20:00,46.5,2.706136,0.0,2.706136,15.500000,12.793864,0.0,640.013207,0.0,0.0,180721,3612.62
8757,12/31/17 21:00,45.0,2.482224,0.0,2.482224,15.000000,12.517776,0.0,626.201901,0.0,0.0,181347,3625.13
8758,12/31/17 22:00,41.7,3.490476,0.0,3.490476,13.900000,10.409524,0.0,520.736568,0.0,0.0,181868,3635.54
